###                                  <center> CSE555 – Deep Learning (Spring 2018) </center>
###                                               <center> Homework 2 </center>
###                                    <center>   Harlinton Palacios Mosquera </center>
                                   
###                                       <center>  ID: 161041033  </center>                                        
                             

In [1]:
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import os
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
import random
import time
import sys

In [2]:
def save_frame(joints, fname, simplified=False):
    fig = plt.figure()
    ax = fig.gca(projection='3d', xlabel='x', ylabel='y', zlabel='z', )
    ax = fig.gca(projection='3d', xlim=[16.4, 16.9], ylim=[3.9, 4.4], zlim=[383, 383.5], xlabel='x', ylabel='y', zlabel='z', )
    ax.view_init(azim=60)
    joints_lines = [[20, 1, 2, 1, 8 , 10, 2, 9 , 11, 3, 4, 7, 7, 5 , 6 , 14, 15, 16, 17],
                    [3 , 3, 3, 8, 10, 12, 9, 11, 13, 4, 7, 5, 6, 14, 15, 16, 17, 18, 19]]


    if(simplified):
        joints = joints.reshape((20, 3))

    [ax.scatter(j[0], j[2]/4, 400-j[1]) for j in joints]

    for i in range(19):
        c1 = joints_lines[0][i] - 1
        c2 = joints_lines[1][i] - 1
        p1x, p1y, p1z = joints[c1]
        p2x, p2y, p2z = joints[c2]
        x = np.array([p1x, p2x])
        y = 400-np.array([p1y, p2y])
        z = np.array([p1z, p2z])/4
        ax.plot(x, z, y)
    #plt.show()
    fig.savefig(fname)
    plt.cla()
    plt.close(fig)

def save_action(action, save_dir, simplified=False):
    for i, joints in enumerate(action):
        save_frame(joints, "{0}/{1}.png".format(save_dir, i), simplified)

def read_action(fname, simplified=False, normed=False):
    data = np.loadtxt(fname)[:,:3]
    if(normed):
        data[:,0] = 1e-4 +data[:,0] / np.linalg.norm(data[:,0])
        data[:,1] = 1e-4 +data[:,1] / np.linalg.norm(data[:,1])
        data[:,2] = 1e-4 +data[:,2] / np.linalg.norm(data[:,2])       
    action = np.split(data, len(data)/20)
    if(simplified):
        action = np.array([joints.reshape((20*3)) for joints in action])
    return action

def read_actions(data_dir, simplified=False, normed=False):
    return [read_action(os.path.join(data_dir, f), simplified, normed) for f in os.listdir(data_dir)]


In this section prepared the data set using a vector (joints_lines) which I used to connect the skeletal extremities. also the function is created to save the frame that will generate the nueronal network.

<img src="https://www.researchgate.net/profile/Xiaoqiang_Li/publication/312642332/figure/fig2/AS:541682212052992@1506158511009/Skeleton-of-MSR-Action-3D.png" alt="Drawing" style="width: 100px;"/>

In [3]:
class ModelNetwork:
    """
    RNN with num_layers LSTM layers and a fully-connected output layer
    The network allows for a dynamic number of iterations, depending on the
    inputs it receives.
       out   (fc layer; out_size)
        ^
       lstm
        ^
       lstm  (lstm size)
        ^
        in   (in_size)
    """
    def __init__(self, in_size, lstm_size, num_layers, out_size, session, learning_rate=0.003, name="rnn"):
        self.scope = name
        self.in_size = in_size
        self.lstm_size = lstm_size
        self.num_layers = num_layers
        self.out_size = out_size

        self.session = session

        self.learning_rate = tf.constant( learning_rate )

        # Last state of LSTM, used when running the network in TEST mode
        self.lstm_last_state = np.zeros((self.num_layers*2*self.lstm_size,))

        with tf.variable_scope(self.scope):
            ## (batch_size, timesteps, in_size)
            self.xinput = tf.placeholder(tf.float32, shape=(None, None, self.in_size), name="xinput")
            self.lstm_init_value = tf.placeholder(tf.float32, shape=(None, self.num_layers*2*self.lstm_size), name="lstm_init_value")

            # LSTM
            self.lstm_cells = [ tf.contrib.rnn.BasicLSTMCell(self.lstm_size, forget_bias=1.0, state_is_tuple=False) for i in range(self.num_layers)]
            self.lstm = tf.contrib.rnn.MultiRNNCell(self.lstm_cells, state_is_tuple=False)

            # Iteratively compute output of recurrent network
            outputs, self.lstm_new_state = tf.nn.dynamic_rnn(self.lstm, self.xinput, initial_state=self.lstm_init_value, dtype=tf.float32)

            # Linear activation (FC layer on top of the LSTM net)
            self.rnn_out_W = tf.Variable(tf.random_normal( (self.lstm_size, self.out_size), stddev=0.01 ))
            self.rnn_out_B = tf.Variable(tf.random_normal( (self.out_size, ), stddev=0.01 ))

            outputs_reshaped = tf.reshape( outputs, [-1, self.lstm_size] )
            network_output = ( tf.matmul( outputs_reshaped, self.rnn_out_W ) + self.rnn_out_B )

            batch_time_shape = tf.shape(outputs)
            self.final_outputs = tf.reshape( tf.nn.softmax( network_output), (batch_time_shape[0], batch_time_shape[1], self.out_size) )


            ## Training: provide target outputs for supervised training.
            self.y_batch = tf.placeholder(tf.float32, (None, None, self.out_size))
            y_batch_long = tf.reshape(self.y_batch, [-1, self.out_size])

            self.cost = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(y_batch_long, network_output))), 'cost')
            tf.summary.scalar('cost', self.cost)
            
            self.train_op = tf.train.RMSPropOptimizer(self.learning_rate, 0.9).minimize(self.cost)


    ## Input: X is a single element, not a list!
    def run_step(self, x, init_zero_state=True):
        ## Reset the initial state of the network.
        if init_zero_state:
            init_value = np.zeros((self.num_layers*2*self.lstm_size,))
        else:
            init_value = self.lstm_last_state

        out, next_lstm_state = self.session.run([self.final_outputs, self.lstm_new_state], feed_dict={self.xinput:[x], self.lstm_init_value:[init_value]   } )

        self.lstm_last_state = next_lstm_state[0]

        return out[0][0]

    def train_batch(self, xbatch, ybatch, merged):
        init_value = np.zeros((xbatch.shape[0], self.num_layers*2*self.lstm_size))

        summary, cost, _ = self.session.run([merged, self.cost, self.train_op], feed_dict={self.xinput:xbatch, self.y_batch:ybatch, self.lstm_init_value:init_value   } )

        return summary, cost

###################################### Load the data ##############################################################
actions = read_actions('./data', simplified=True, normed=True)

in_size = out_size = 20*3 # frame (x y z for 20 joints)
lstm_size = 256 #128
num_layers = 2
batch_size = 1 #128
time_steps = 10 #50

NUM_TRAIN_BATCHES = 30000

LEN_TEST_TEXT = 200 # Number of test characters of text to generate after training the network

## Initialize the network
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.InteractiveSession(config=config)


net = ModelNetwork(in_size = in_size,
                    lstm_size = lstm_size,
                    num_layers = num_layers,
                    out_size = out_size,
                    session = sess,
                    learning_rate = 0.003,
                    name = "msr_rnn_network")

sess.run(tf.global_variables_initializer())

ckpt_file = "./Training_model/model.ckpt"
saver = tf.train.Saver(tf.global_variables())

################################################# 1) TRAIN THE NETWORK #########################################################################
if True:

    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter('./logs', sess.graph)

    last_time = time.time()

    for i in range(NUM_TRAIN_BATCHES):

        # data = random.choice(actions)
        data = actions[0]
        time_steps = len(data) - 1

        batch = np.zeros((batch_size, time_steps, in_size))
        batch_y = np.zeros((batch_size, time_steps, in_size))

        possible_batch_ids = range(len(data)-time_steps-1)
        # Sample time_steps consecutive samples from the dataset text file
        # batch_id = random.sample( possible_batch_ids, batch_size )
        batch_id = [0]

        for j in range(time_steps):
            ind1 = [k+j for k in batch_id]
            ind2 = [k+j+1 for k in batch_id]
            batch[:, j, :] = data[ind1, :]
            batch_y[:, j, :] = data[ind2, :]
        
        summary, cst = net.train_batch(batch, batch_y, merged)
        train_writer.add_summary(summary, i)

        
        
        
        if (i%100) == 0:
            new_time = time.time()
            diff = new_time - last_time
            last_time = new_time
            print("batch: ",i,"   loss: ",cst,"   speed: ",(100.0/diff)," batches / s")

            saver.save(sess, ckpt_file)
        
        
    print ("Loss: " , cst)
############################### GENERATE LEN_TEST_TEXT CHARACTERS USING THE TRAINED NETWORK########################################

TEST_PREFIX = actions[0][:1]
for i in range(len(TEST_PREFIX)):
    out = net.run_step( [TEST_PREFIX[i]] , i==0)
    save_frame(1000*out, './skeleton_action/{0}.png'.format(i), True)

print("PRED:")
gen_str = TEST_PREFIX
for j in range(LEN_TEST_TEXT):   
    out = net.run_step( [out] , False )
    save_frame(1000*out, './skeleton_action/{0}.png'.format(i+j), True)

print(gen_str)


batch:  0    loss:  1.82675    speed:  201.4594907968889  batches / s
batch:  100    loss:  0.161753    speed:  5.454073450251663  batches / s
batch:  200    loss:  0.461087    speed:  5.573392707322988  batches / s
batch:  300    loss:  0.139019    speed:  5.521544990609644  batches / s
batch:  400    loss:  0.198992    speed:  5.524571768383797  batches / s
batch:  500    loss:  0.306895    speed:  5.448751258552127  batches / s
batch:  600    loss:  0.336003    speed:  5.355743866693985  batches / s
batch:  700    loss:  0.113127    speed:  5.091218333466654  batches / s
batch:  800    loss:  0.119345    speed:  5.219747189651741  batches / s
batch:  900    loss:  0.157287    speed:  5.243490997788517  batches / s
batch:  1000    loss:  0.189803    speed:  5.258978192407551  batches / s
batch:  1100    loss:  0.105675    speed:  5.171756645031409  batches / s
batch:  1200    loss:  0.133948    speed:  4.873671200211879  batches / s
batch:  1300    loss:  0.128014    speed:  4.870456

batch:  10500    loss:  0.0851422    speed:  3.1659914135689435  batches / s
batch:  10600    loss:  0.0844953    speed:  3.245289230372149  batches / s
batch:  10700    loss:  0.0854549    speed:  2.8369855059351603  batches / s
batch:  10800    loss:  0.0848452    speed:  2.9510359277991682  batches / s
batch:  10900    loss:  0.0731993    speed:  3.2203328225347625  batches / s
batch:  11000    loss:  0.0791754    speed:  3.265002805137839  batches / s
batch:  11100    loss:  0.0810935    speed:  3.245664015461922  batches / s
batch:  11200    loss:  0.0821387    speed:  3.26719808769504  batches / s
batch:  11300    loss:  0.0473684    speed:  3.2722085290229397  batches / s
batch:  11400    loss:  0.0720018    speed:  3.249003329692684  batches / s
batch:  11500    loss:  0.0780511    speed:  4.6650005494936755  batches / s
batch:  11600    loss:  0.0817832    speed:  4.7775092170825655  batches / s
batch:  11700    loss:  0.0838093    speed:  4.714796347307105  batches / s
batch:

batch:  21300    loss:  0.0850408    speed:  3.825056005330833  batches / s
batch:  21400    loss:  0.0841928    speed:  3.8482878177066246  batches / s
batch:  21500    loss:  0.0894734    speed:  3.8516525183335104  batches / s
batch:  21600    loss:  0.0877128    speed:  3.7010750190681114  batches / s
batch:  21700    loss:  0.0864099    speed:  3.8499441656282714  batches / s
batch:  21800    loss:  0.0850193    speed:  3.8340573164650085  batches / s
batch:  21900    loss:  0.084479    speed:  3.76318501938151  batches / s
batch:  22000    loss:  0.0841907    speed:  3.246365149190743  batches / s
batch:  22100    loss:  0.0843619    speed:  3.783037917886031  batches / s
batch:  22200    loss:  0.0838551    speed:  3.537359764688336  batches / s
batch:  22300    loss:  0.0591513    speed:  3.93634226524827  batches / s
batch:  22400    loss:  0.066007    speed:  4.663147686948429  batches / s
batch:  22500    loss:  0.0708234    speed:  4.107608164855258  batches / s
batch:  226

C:\Users\Harlinton\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Requested projection is different from current axis projection, creating new axis with requested projection.
  after removing the cwd from sys.path.


PRED:
[[ 0.02579736  0.01654485  0.03081695  0.03393485  0.01695598  0.03086287
   0.0298661   0.01551705  0.02976091  0.0298661   0.02045051  0.03196482
   0.02665394  0.02497284  0.03044963  0.03307827  0.0251784   0.03031189
   0.03008025  0.02476728  0.03178116  0.02451249  0.02230055  0.03072512
   0.03521972  0.02271168  0.03090878  0.02494078  0.02867294  0.03017415
   0.03479143  0.02908406  0.03012823  0.02472663  0.03011186  0.03031189
   0.03479143  0.03052298  0.03008232  0.02729637  0.03751204  0.03113836
   0.03286413  0.03751204  0.03113836  0.02686808  0.04594003  0.03214848
   0.03307827  0.04635115  0.03155159  0.0260115   0.04799564  0.03022006
   0.03286413  0.0482012   0.02962317  0.03008025  0.00955579  0.02934768]]


##  <center>Results and configuration</center>

<b>MSR Action3D Dataset : </b>
20 action types, 10 subjects, each subject performs each action 2 or 3 times. There are 567 depth map sequences in total. The resolution is 640x240. The data was recorded with a depth sensor similar to the Kinect device

<b>Parameters used: </b>



<b>Number of test characters:</b>  200

<b>LSTM_size:</b> 256

<b>Learning rate:</b> 0.003

<b>Numero of layers: </b> 2

<b>Loss function:</b> Euclidean Distance

<b>Optimizer:</b> RMSProp

<b>Input size</b> = Output size= 60 (20*3)(x y z for 20 joints)

<b> NUM_TRAIN_BATCHES: </b> = 30000

##  After training and evaluation of the neural network with the parameters established in this homework, we obtained:

<b>Loss:</b>  0.086614 and the neural network generated 200 frames with which I created the following video: https://www.youtube.com/watch?v=-0TdVNibr6k


In [ ]:
import cv2
import numpy as np
import os
framerate = 10 
from os.path import isfile, join
 
def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
 
 
    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        print(filename)
        #inserting the frames into an image array
        frame_array.append(img)
     
    #out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Be sure to use lower case
    out = cv2.VideoWriter(pathOut, fourcc, framerate, (size))
    #out = cv2.VideoWriter(pathOut, -1, 1, (size))
 
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()
pathIn = "./skeleton_action/"
pathOut = 'Video.mp4'
fps = 25.0
convert_frames_to_video(pathIn, pathOut, fps)

This code generated a vídeo with the frame generated before 
:https://www.youtube.com/watch?v=-0TdVNibr6k